In [ ]:
# !pip install PyPDF2

In [ ]:
pip install sentence-transformers faiss-cpu

In [ ]:
!pip install langchain

In [ ]:
!pip install pymupdf

In [ ]:
# import PyPDF2
# from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import fitz
import re


In [ ]:
# def extract_text_from_pdf(path):
#     with open(path,'r') as file:
#         pdf_reader=PyPDF2.PdfReader(file)
#         text=""
#         for page in pdf_reader.pages:
#             text+=page.extract_text()
#             return text
# path=r'/kaggle/input/food-safety-manual/fpc-manual.pdf'
# pdf_text=extract_text_from_pdf(path)

In [ ]:
def extract_text_from_pdf(path):
    text=""
    pdf_doc=fitz.open(path)
    for page in pdf_doc:
        text+=page.get_text()
    pdf_doc.close()
    return text
path=r'/kaggle/input/food-safety-manual/fpc-manual.pdf'
pdf_text=extract_text_from_pdf(path)

In [ ]:
def clean_text(text):
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces/newlines
    text = re.sub(r'[^a-zA-Z0-9\s,.]', '', text)  # Remove special characters
    return text.strip()

In [ ]:
cleaned_text=clean_text(pdf_text)
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)
chunks=text_splitter.split_text(cleaned_text)

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model=SentenceTransformer('all-MiniLm-L6-v2')
chunk_embeddings=embedding_model.encode(chunks)

In [ ]:
dimension=len(chunk_embeddings[0])
index=faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))

In [ ]:
def retrieve_similar_chunks(query, k=5):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [chunks[i] for i in indices[0]]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
 
model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")
 


In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
import time

In [ ]:
def generate_answer(query):
    start_time=time.time()
    retrieved_chunks = retrieve_similar_chunks(query)
    context = " ".join(retrieved_chunks)
    prompt = f"context:{context}\n\nQuestion: {query}\n\nAnswer:"
    
    inputs = tokenizer(prompt, return_tensors="pt",padding=True, truncation=True).to("cuda")
    outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=200,temperature=0.7,do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    end_time = time.time()
    print(end_time-start_time)
    return response

In [ ]:
query='how to make margarita cheese pizza?'
print(generate_answer(query))